In [3]:


import tensorflow as tf
from tensorflow.keras.applications import VGG19, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import os
import re

# Updated dataset paths for rose
train_dir = 'C:/Users/HP/Desktop/train_test_split_ext/sent3/images_rose_ext/train'
test_dir = 'C:/Users/HP/Desktop/train_test_split_ext/sent3/images_rose_ext/test'
validation_dir = 'C:/Users/HP/Desktop/train_test_split_ext/sent3/images_rose_ext/val'

# Data augmentation for training data
train_datagen = ImageDataGenerator(rescale=1./255)

# Only rescaling for validation and test data
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Define checkpoint path for rose
checkpoint_dir = "./checkpoints_rose"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "model_checkpoint_rose_epoch_{epoch:02d}.h5")

# Load base models without the top layers
input_tensor = Input(shape=(224, 224, 3))
vgg19_base = VGG19(weights='imagenet', include_top=False, input_tensor=input_tensor)
densenet_base = DenseNet121(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze all base model layers initially
for layer in vgg19_base.layers:
    layer.trainable = False
for layer in densenet_base.layers:
    layer.trainable = False

# Extract features
vgg19_output = GlobalAveragePooling2D()(vgg19_base.output)
densenet_output = GlobalAveragePooling2D()(densenet_base.output)

# Concatenate features
merged = Concatenate()([vgg19_output, densenet_output])
merged = Dense(1024, activation='relu')(merged)
predictions = Dense(1, activation='sigmoid')(merged)

# Define final model
model = Model(inputs=input_tensor, outputs=predictions)

# Find latest checkpoint for rose
def get_latest_checkpoint():
    checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.startswith("model_checkpoint_rose_epoch")]
    if not checkpoint_files:
        return None, 0

    # Extract epoch numbers
    epochs_trained = [int(re.search(r'epoch_(\d+)', f).group(1)) for f in checkpoint_files]
    latest_epoch = max(epochs_trained)
    latest_checkpoint = os.path.join(checkpoint_dir, f"model_checkpoint_rose_epoch_{latest_epoch:02d}.h5")
    return latest_checkpoint, latest_epoch

latest_checkpoint, last_trained_epoch = get_latest_checkpoint()

# Load the latest model checkpoint if available
if latest_checkpoint and os.path.exists(latest_checkpoint):
    print(f"Resuming training from checkpoint: {latest_checkpoint}")
    model = load_model(latest_checkpoint)
else:
    print("No saved model found. Starting fresh.")

# Compile model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# ModelCheckpoint callback for rose
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)

# Training Phases
epochs_phase_1 = 10
epochs_phase_2 = 5
epochs_phase_3 = 5

# Determine phase and continue training
if last_trained_epoch < epochs_phase_1:
    print(f"Starting Phase 1 (continuing from epoch {last_trained_epoch + 1})")
    history1 = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        initial_epoch=last_trained_epoch,
        epochs=epochs_phase_1,
        callbacks=[checkpoint_callback]
    )
    last_trained_epoch = epochs_phase_1

# Unfreeze top 5 layers and continue training
if last_trained_epoch < epochs_phase_1 + epochs_phase_2:
    for layer in vgg19_base.layers[-5:]:
        layer.trainable = True
    for layer in densenet_base.layers[-5:]:
        layer.trainable = True

    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    print(f"Starting Phase 2 (continuing from epoch {last_trained_epoch + 1})")
    history2 = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        initial_epoch=last_trained_epoch,
        epochs=epochs_phase_1 + epochs_phase_2,
        callbacks=[checkpoint_callback]
    )
    last_trained_epoch = epochs_phase_1 + epochs_phase_2

# Unfreeze all layers and fine-tune
if last_trained_epoch < epochs_phase_1 + epochs_phase_2 + epochs_phase_3:
    for layer in vgg19_base.layers:
        layer.trainable = True
    for layer in densenet_base.layers:
        layer.trainable = True

    model.compile(
        optimizer=Adam(learning_rate=1e-6),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    print(f"Starting Phase 3 (continuing from epoch {last_trained_epoch + 1})")
    history3 = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        initial_epoch=last_trained_epoch,
        epochs=epochs_phase_1 + epochs_phase_2 + epochs_phase_3,
        callbacks=[checkpoint_callback]
    )

# Evaluate model
test_loss, test_accuracy = model.evaluate(test_generator)
val_loss, val_accuracy = model.evaluate(validation_generator)

print(f'Test Loss (Rose): {test_loss}')
print(f'Test Accuracy (Rose): {test_accuracy}')
print(f'Validation Loss (Rose): {val_loss}')
print(f'Validation Accuracy (Rose): {val_accuracy}')

Found 2238 images belonging to 2 classes.
Found 482 images belonging to 2 classes.
Found 480 images belonging to 2 classes.
No saved model found. Starting fresh.
Starting Phase 1 (continuing from epoch 1)
Epoch 1/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.7369 - loss: 0.5188 
Epoch 1: saving model to ./checkpoints_rose\model_checkpoint_rose_epoch_01.h5


69/69 ━━━━━━━━━━━━━━━━━━━━ 919s 13s/step - accuracy: 0.7385 - loss: 0.5168 - val_accuracy: 0.9438 - val_loss: 0.2213
Epoch 2/10
 1/69 ━━━━━━━━━━━━━━━━━━━━ 11:40 10s/step - accuracy: 1.0000 - loss: 0.1486

C:\Users\HP\Documents\anaconda2\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: saving model to ./checkpoints_rose\model_checkpoint_rose_epoch_02.h5


69/69 ━━━━━━━━━━━━━━━━━━━━ 168s 2s/step - accuracy: 1.0000 - loss: 0.1486 - val_accuracy: 0.9417 - val_loss: 0.2187
Epoch 3/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.9506 - loss: 0.1947 
Epoch 3: saving model to ./checkpoints_rose\model_checkpoint_rose_epoch_03.h5


...

69/69 ━━━━━━━━━━━━━━━━━━━━ 4834s 68s/step - accuracy: 0.9361 - loss: 0.1654 - val_accuracy: 0.9812 - val_loss: 0.0592
Epoch 17/20
 1/69 ━━━━━━━━━━━━━━━━━━━━ 1:32:59 82s/step - accuracy: 0.9688 - loss: 0.0931
Epoch 17: saving model to ./checkpoints_rose\model_checkpoint_rose_epoch_17.h5


69/69 ━━━━━━━━━━━━━━━━━━━━ 422s 5s/step - accuracy: 0.9688 - loss: 0.0931 - val_accuracy: 0.9812 - val_loss: 0.0595
Epoch 18/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 81s/step - accuracy: 0.9558 - loss: 0.1115  
Epoch 18: saving model to ./checkpoints_rose\model_checkpoint_rose_epoch_18.h5


69/69 ━━━━━━━━━━━━━━━━━━━━ 5952s 86s/step - accuracy: 0.9558 - loss: 0.1114 - val_accuracy: 0.9417 - val_loss: 0.1296
Epoch 19/20
 1/69 ━━━━━━━━━━━━━━━━━━━━ 1:33:48 83s/step - accuracy: 1.0000 - loss: 0.0287
Epoch 19: saving model to ./checkpoints_rose\model_checkpoint_rose_epoch_19.h5


69/69 ━━━━━━━━━━━━━━━━━━━━ 425s 5s/step - accuracy: 1.0000 - loss: 0.0287 - val_accuracy: 0.9375 - val_loss: 0.1408
Epoch 20/20
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 79s/step - accuracy: 0.9736 - loss: 0.0896  
Epoch 20: saving model to ./checkpoints_rose\model_checkpoint_rose_epoch_20.h5


69/69 ━━━━━━━━━━━━━━━━━━━━ 5785s 84s/step - accuracy: 0.9736 - loss: 0.0896 - val_accuracy: 0.9646 - val_loss: 0.0877
15/15 ━━━━━━━━━━━━━━━━━━━━ 327s 22s/step - accuracy: 0.9504 - loss: 0.1186
16/16 ━━━━━━━━━━━━━━━━━━━━ 328s 20s/step - accuracy: 0.9759 - loss: 0.0769
Test Loss (Rose): 0.10316033661365509
Test Accuracy (Rose): 0.9583333134651184
Validation Loss (Rose): 0.087323859333992
Validation Accuracy (Rose): 0.9647302627563477
